<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/pl_whisper_tiny_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-h95vj2gl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-h95vj2gl
  Resolved https://github.com/huggingface/transformers to commit 235e5d4991e8a0984aa78db91087b49622c7740e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8125679 sha256=2c4c5faab47e04abe4d47698fafce649957a786c204885bdae80ccfbb3d39475
  Stored in directory: /tmp/pip-ephem-wheel-cache-alg023hm/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "pl", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1831
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=5d445683d98d44406a56be95306d73de784e7e1410f336248b136785dd815694
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [19]:
model = whisper.load_model("tiny")

100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 58.4MiB/s]


In [20]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [21]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [22]:
text

,0
0,to jest niezwykle istotne dajemy tutaj przykła...
1,do których będzie musiała również się odnieść ...
2,ogromne pieniądze szacowane na setki miliardów...
3,czy też odwrotnie na zasadzie deregulacji zmni...
4,tym bardziej gdy wsparcie to odbywa się poprze...
...,...
1826,w pierwszej sprawie uważam że poza podstawową ...
1827,prawda? jak się działy w polsce różne niedobre...
1828,ale nie wyobrażam sobie że w imieniu prezydenc...
1829,podkreślę jeszcze raz propozycje w tej materii...


In [23]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [24]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,to jest niezwykle istotne dajemy tutaj przykła...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.027740478515625, -0.004241943359375, 0.001...",16000
1,male,do których będzie musiała również się odnieść ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.028228759765625, 0.04449462890625, 0.041656...",16000
2,male,ogromne pieniądze szacowane na setki miliardów...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00189208984375, -0.0059814453125, -0.00433...",16000
3,male,czy też odwrotnie na zasadzie deregulacji zmni...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00128173828125, 0.002105712890625, -0.0006...",16000
4,male,tym bardziej gdy wsparcie to odbywa się poprze...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001953125, 0.000823974609375, 0.0057678222...",16000
...,...,...,...,...,...
1826,male,w pierwszej sprawie uważam że poza podstawową ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, 0.0003662109375, 0.0002136...",16000
1827,male,prawda? jak się działy w polsce różne niedobre...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.009368896484375, 0.022918701171875, -0.004...",16000
1828,female,ale nie wyobrażam sobie że w imieniu prezydenc...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001129150390625, 6.103515625e-05, -0.000183...",16000
1829,male,podkreślę jeszcze raz propozycje w tej materii...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000518798828125, 0.000213623046875, 0.0, 0....",16000


In [25]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,to jest niezwykle istotne dajemy tutaj przykła...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.027740478515625, -0.004241943359375, 0.001...",16000
1,male,do których będzie musiała również się odnieść ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.028228759765625, 0.04449462890625, 0.041656...",16000
2,male,ogromne pieniądze szacowane na setki miliardów...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00189208984375, -0.0059814453125, -0.00433...",16000
3,male,czy też odwrotnie na zasadzie deregulacji zmni...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00128173828125, 0.002105712890625, -0.0006...",16000
4,male,tym bardziej gdy wsparcie to odbywa się poprze...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001953125, 0.000823974609375, 0.0057678222...",16000
...,...,...,...,...,...
1824,male,czy unia europejska jest przedsiębiorstwem i d...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000946044921875, 0.0047607421875, 0.003082...",16000
1825,male,nienawiść rasowa lub nienawiść klasowa są tylk...,/root/.cache/huggingface/datasets/downloads/ex...,"[3.0517578125e-05, 0.00225830078125, 0.0021362...",16000
1826,male,w pierwszej sprawie uważam że poza podstawową ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, 0.0003662109375, 0.0002136...",16000
1827,male,prawda? jak się działy w polsce różne niedobre...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.009368896484375, 0.022918701171875, -0.004...",16000


In [26]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
12,female,prawo do tej opieki gwarantują obywatelom zaró...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00439453125, -6.103515625e-05, -0.002685546...",16000
13,female,sacharowa ale także nagrodą pro dignitate huma...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.01800537109375, -0.029296875, -0.035675048...",16000
16,female,gdzie jest mechanizm wiążący praworządność z f...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0013427734375, 6.103515625e-05, -0.001...",16000
22,female,pani przewodnicząca panie komisarzu panie posł...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.010406494140625, -0.00653076171875, -0.000...",16000
26,female,europejska polityka sąsiedztwa powstała w celu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002410888671875, 0.00238037109375, -0.00112...",16000
...,...,...,...,...,...
1794,female,nowy projekt przepisów którego celem jest zwię...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002655029296875, -0.0008544921875, 0.000274...",16000
1818,female,panie komisarzu stwierdził pan że krytyka syst...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.004058837890625, 0.00689697265625, 0.00457...",16000
1819,female,kandydatem na nowego ministra edukacji jest os...,/root/.cache/huggingface/datasets/downloads/ex...,"[9.1552734375e-05, 0.0008544921875, -0.0003662...",16000
1828,female,ale nie wyobrażam sobie że w imieniu prezydenc...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001129150390625, 6.103515625e-05, -0.000183...",16000


In [27]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="pl")
    return result["text"]

In [28]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [29]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.5
Processed 2 files, WER: 0.36
Processed 3 files, WER: 0.4
Processed 4 files, WER: 0.3
Processed 5 files, WER: 0.1875
Processed 6 files, WER: 0.47619047619047616
Processed 7 files, WER: 0.6086956521739131
Processed 8 files, WER: 0.3
Processed 9 files, WER: 0.625
Processed 10 files, WER: 0.631578947368421
Processed 11 files, WER: 0.35
Processed 12 files, WER: 0.3
Processed 13 files, WER: 0.6923076923076923
Processed 14 files, WER: 0.125
Processed 15 files, WER: 0.6
Processed 16 files, WER: 0.5384615384615384
Processed 17 files, WER: 0.5
Processed 18 files, WER: 0.2857142857142857
Processed 19 files, WER: 0.25
Processed 20 files, WER: 0.5833333333333334
Processed 21 files, WER: 0.23529411764705882
Processed 22 files, WER: 0.5
Processed 23 files, WER: 0.4583333333333333
Processed 24 files, WER: 0.25
Processed 25 files, WER: 0.2
Processed 26 files, WER: 0.2857142857142857
Processed 27 files, WER: 0.36666666666666664
Processed 28 files, WER: 0.37037037037037035
Pro

In [30]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.4666666666666667
Processed 2 files, WER: 0.5263157894736842
Processed 3 files, WER: 0.75
Processed 4 files, WER: 1.125
Processed 5 files, WER: 0.6666666666666666
Processed 6 files, WER: 0.5
Processed 7 files, WER: 0.3333333333333333
Processed 8 files, WER: 0.3333333333333333
Processed 9 files, WER: 0.46153846153846156
Processed 10 files, WER: 0.38095238095238093
Processed 11 files, WER: 0.45454545454545453
Processed 12 files, WER: 0.4857142857142857
Processed 13 files, WER: 0.8
Processed 14 files, WER: 0.14285714285714285
Processed 15 files, WER: 0.6956521739130435
Processed 16 files, WER: 0.45161290322580644
Processed 17 files, WER: 0.35294117647058826
Processed 18 files, WER: 0.0
Processed 19 files, WER: 0.25
Processed 20 files, WER: 0.4375
Processed 21 files, WER: 0.3333333333333333
Processed 22 files, WER: 1.0
Processed 23 files, WER: 0.5862068965517241
Processed 24 files, WER: 0.4166666666666667
Processed 25 files, WER: 0.5
Processed 26 files, WER: 1.5
Pr